In [1]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
Train=pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/train.csv')
Test= pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/test.csv')
sample_submission= pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/sample_submission.csv')

In [2]:
#replace ".."with .100
Train=Train.replace('..', .100)  
Test=Test.replace('..', 0.100)  

In [3]:
#Unique country for iteration
unique = Train['Country Name'].unique()

In [4]:
#trainig and inferencing with SVC and ARIMA
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
from sklearn.svm import SVC
import warnings

# Ignore all warnings

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
warnings.filterwarnings("ignore")
j=0
for i in unique:
    transposed_df=0
    #creating traindata
    fg = Train[Train['Country Name'] == i].reset_index(drop=True)
    columns_to_drop = ['Country Code', 'Country Name','Indicator']
    fg = fg.drop(columns_to_drop, axis=1)
    transposed_df = fg.transpose()
    #print(fg)
    #print('done for now')
    Xtrain = transposed_df[[0,1, 2,3,4,5,6,7,8,9,10]]
    Xtrain
    Ytrain=transposed_df[11]
    Ytrain
    svc = SVC()
    try:
        svc.fit(Xtrain,Ytrain)
        #creating testdata
        fgt = Test[Test['Country Name'] == i].reset_index(drop=True)
        columns_to_drop = [ 'Country Name','Indicator']
        fgt = fgt.drop(columns_to_drop, axis=1)
        transposed_df = fgt.transpose()
        #print(transposed_df)
        
        Xtest = transposed_df[[0,1, 2,3,4,5,6,7,8,9,10]]
        forecast = svc.predict(Xtest.astype(float))
        
        model = ARIMA(Ytrain.astype(float), order=(0,1,0))  # Adjust order as needed
        model_fit = model.fit()

    
        forecast2 = model_fit.forecast(steps=16)
        
        row_index = sample_submission[sample_submission.eq(i).any(axis=1)].index[0]


        new_values = [i, forecast[0], forecast[1],forecast[2], forecast[3], forecast[4],forecast2[31]]
        sample_submission.loc[row_index] = new_values
    except:
        j+=1
    
    



In [5]:
sample_submission.head()

,Country Name,2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2030 [YR2030]
0,Afghanistan,0.297971775,0.297971775,0.239506901,0.239506901,0.297971775,0.297972
1,Albania,1.665218525,1.665218525,1.665218525,1.665218525,1.665218525,1.665219
2,Algeria,3.951960939,3.951960939,3.951960939,3.951960939,3.951960939,3.951961
3,American Samoa,3.5276,3.1392,1.1129,1.6479,3.1007,0.684000
4,Andorra,7.206273645,7.206273645,6.805400082,7.206273645,7.286756008,6.485769


In [6]:
sample_submission.to_csv('submission.csv', index=False)